## Linear Algebra: Solving equations
__CYBR 304 & MATH 420__ <br>
_Spring 2024_ <br>


Let's solve a $3 \times  3$ linear system of equations. The coefficient matrix is a Julia array. To enter the matrix by hand, we write the members of each row separated with spaces (_not_ commas) and separate each row with a semicolon. Our coefficient matrix is

In [1]:
Mat = [1 2 3; 4 5 6; 7 8 19]

3×3 Matrix{Int64}:
 1  2   3
 4  5   6
 7  8  19

Similarly, we can enter the right-hand side (RHS) by hand too. We'll assign `b` to the RHS.

In [2]:
b = [2; 5; 10]

3-element Vector{Int64}:
  2
  5
 10

We can solve the linear equation `Mat x = b` with the backslash command

In [3]:
x  = Mat \ b

3-element Vector{Float64}:
 0.19999999999999993
 0.6000000000000002
 0.19999999999999996

The result is a column vector of the solution.  

Notice that the entries of both the coefficient matrix and the right-hand side are `Int64` numbers, but each member of the solution are `Float64` numbers.

Let's check our work.  In Julia, the asterisk also means matrix multiplication. Let's find `b - Mat * x`. This difference is called the _residual_. For the true solution, the residual is the zero vector; let's check:

In [4]:
b - Mat * x 

3-element Vector{Float64}:
 0.0
 0.0
 0.0

Super, the residual is the zero vector. Almost surely, the solution is correct.

If we need to do exact rational arithmetic, we can do that. Actually, instead of using the default of `Int64` numbers, let's convert to `BigInt` numbers:

In [5]:
Mat = map(x -> convert(BigInt, x)//1,Mat)

3×3 Matrix{Rational{BigInt}}:
 1  2   3
 4  5   6
 7  8  19

In [6]:
b = map(x -> convert(BigInt, x) //1,b)

3-element Vector{Rational{BigInt}}:
  2
  5
 10

In [7]:
x = Mat \ b

3-element Vector{Rational{BigInt}}:
 1//5
 3//5
 1//5

For larger or more complex coefficient matrices, we can define a function that evaluates the $i,j$ matrix element; a famous example:

In [8]:
function F(i,j) 
   if i == j  
      2
    elseif abs(i - j) == 1
      1
    elseif abs(i-j) == 2
      -2
    else 
       0
  end
end

F (generic function with 1 method)

In [9]:
n = 9;

An array comprehension generates our array:

In [10]:
M = [F(i,j) for i=1:n, j=1:n]

9×9 Matrix{Int64}:
  2   1  -2   0   0   0   0   0   0
  1   2   1  -2   0   0   0   0   0
 -2   1   2   1  -2   0   0   0   0
  0  -2   1   2   1  -2   0   0   0
  0   0  -2   1   2   1  -2   0   0
  0   0   0  -2   1   2   1  -2   0
  0   0   0   0  -2   1   2   1  -2
  0   0   0   0   0  -2   1   2   1
  0   0   0   0   0   0  -2   1   2

The matrix is a _tridiagonal matrix_. Let's solve $M x = b$, where $b$ is a column vector with each member equal to $1$

In [11]:
b = [1  for i=1:n]

9-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [12]:
x  = M  \ b

9-element Vector{Float64}:
 1.8043478260869557
 1.3478260869565222
 1.9782608695652166
 2.739130434782608
 1.717391304347826
 2.739130434782608
 1.978260869565217
 1.3478260869565215
 1.8043478260869563

Checking our work, our solution looks OK-- some members of the residual are about equal to the machine epsilon--not surprising. One member is about ten times the machine epsilon.

In [13]:
M * x - b

9-element Vector{Float64}:
  4.440892098500626e-16
  8.881784197001252e-16
  4.440892098500626e-16
 -1.7763568394002505e-15
  4.440892098500626e-16
  0.0
 -4.440892098500626e-16
  2.220446049250313e-16
  0.0

Julia has a function that computes the matrix inverse. But solving using the matrix inverse is _slower_ than using the backslash. Let's use a $4096 \times 4096$ matrix.

In [14]:
using BenchmarkTools

In [15]:
n = 2^12

4096

In [16]:
M = [F(i,j) for i=1:n, j=1:n];

In [17]:
x = [1  for i=1:n];

In [18]:
b = M * x;

In [19]:
 @btime M \ b;

  815.951 ms (6 allocations: 128.06 MiB)


In [20]:
@btime inv(M) * b;

  2.958 s (10 allocations: 130.09 MiB)


Solving using the matrix inverse is slower and uses more memory--using a matrix inverse is almost always the wrong thing to do. 

Are the two solutions nearly equal? Let's hope so. The Julia function `findmax` will find the maximum member of an array and also find its position.  We might like the member with the greatest magnitude, so we'll map the absolute value function onto the difference:

In [21]:
findmax(map(abs, M \ b  - inv(M)*b))

(5.262457136723242e-14, 2667)

The member with the greatest magnitude is at location 2667 and its magnitude is about $5.3 \times 10^{-14}$. So yes, I'd say the answers are about the same.

We can solve multiple RHS at the same time; for example, let's try three right-hand-sides all at once.

In [22]:
Mat = [1 2 3; 4 5 6; 7 8 107]

3×3 Matrix{Int64}:
 1  2    3
 4  5    6
 7  8  107

In [23]:
b = [1 0 0; 0 1 0 ; 0 0 1]

3×3 Matrix{Int64}:
 1  0  0
 0  1  0
 0  0  1

Each column is a solution. The first column is the solution with the RHS [1,0,1].

In [24]:
x = Mat \ b

3×3 Matrix{Float64}:
 -1.65646     0.646259    0.0102041
  1.31293    -0.292517   -0.0204082
  0.0102041  -0.0204082   0.0102041

Mat * x should be the $3 \times 3 $ identity matrix; let's check:

In [25]:
Mat * x

3×3 Matrix{Float64}:
  1.0          -4.85723e-17  -6.93889e-18
 -2.498e-16     1.0          -6.93889e-18
  2.22045e-16   0.0           1.0

Is it the $3 \times 3 $ identity matrix? Well, almost--the off-diagonal terms differ from zero by an amount that is about the same or smaller than the machine epsilon. We're not surprised by this.  

Let's try another example. A famous matrix that has a determinant that is zero is 
$$
 \begin{bmatrix} 1&2&3\\4&5& 6 \\ 7 & 8 & 9 \end{bmatrix}
$$
Let's change the $3,3$ element just a bit and use it as a coefficient matrix. For just a bit, we'll change
the $3,3$ element to $9 + 2^{-46}$.

In [26]:
Mat = [1 2 3; 4 5 6; 7 8 9+ 2.0^(-46)]

3×3 Matrix{Float64}:
 1.0  2.0  3.0
 4.0  5.0  6.0
 7.0  8.0  9.0

In [27]:
b = [1 0 0; 0 1 0 ; 0 0 1]

3×3 Matrix{Int64}:
 1  0  0
 0  1  0
 0  0  1

As a matrix element, $9+ 2^{-46}$ prints as $9.0$, but its value isn't 9:

In [28]:
9+ 2.0^(-46)

9.000000000000014

In [29]:
x = Mat \ b

3×3 Matrix{Float64}:
  7.03687e13  -1.40737e14   7.03687e13
 -1.40737e14   2.81475e14  -1.40737e14
  7.03687e13  -1.40737e14   7.03687e13

Should we check our work? Always!

In [30]:
Mat * x

3×3 Matrix{Float64}:
  1.0     0.0   0.0
 -0.0625  1.0  -0.0625
  0.0     0.0   0.875

Yikes! This should be the $3 \times 3$ identity matrix, but the off diagonal terms differ a _great deal_ from zero. What's the story? Let's try solving with exact rational numbers

In [31]:
MMat = map(x -> BigInt(x)//1, [1 2 3; 4 5 6; 7 8 9]) + [0 0 0; 0 0 0; 0 0 1//BigInt(2)^46]

3×3 Matrix{Rational{BigInt}}:
 1  2                 3
 4  5                 6
 7  8  633318697598977//70368744177664

In [32]:
b = map(x -> BigInt(x)//1, b)

3×3 Matrix{Rational{BigInt}}:
 1  0  0
 0  1  0
 0  0  1

In [33]:
xx = MMat \ b

3×3 Matrix{Rational{BigInt}}:
  211106232532987//3  -422212465065982//3    70368744177664
 -422212465065980//3   844424930131967//3  -140737488355328
    70368744177664     -140737488355328      70368744177664

Checking this, all is well!

In [34]:
MMat * xx

3×3 Matrix{Rational{BigInt}}:
 1  0  0
 0  1  0
 0  0  1

Let's compare the two solutions

In [35]:
xx = map(x -> convert(Float64,x), xx)

3×3 Matrix{Float64}:
  7.03687e13  -1.40737e14   7.03687e13
 -1.40737e14   2.81475e14  -1.40737e14
  7.03687e13  -1.40737e14   7.03687e13

In [36]:
findmax(map(abs, x - xx))

(0.046875, CartesianIndex(2, 1))

The absolute values of the differences is as large as about $0.046875.$

What we are seeing is something like the butterfly effect. A tiny change in the input to a linear system, can cause a big change in the output. Whence the changes in the input? We need to round the matrix elements to floating point numbers. Plus every arithmetic operation involves more rounding errors.

The matrix condition number (which we will learn about) is a measure of how much the solution to a linear system might change due to a change in the inputs. Julia gives us a quick way to compute the matrix condition number>

In [37]:
using LinearAlgebra

In [38]:
cond(Mat,Inf)

1.3510798882111456e16

I know what you are thinking: The condition number of our matrix is big because its determinate is small:

In [39]:
det(Mat)

-4.263256414560601e-14

Nice guess, but no. Here is a matrix whose determinant is about one, but it has a big condition number.

In [40]:
MM = [6.086956521743044e5 9.130434782599565e5; 2.608695652171305e5 3.913043478266957e5]

2×2 Matrix{Float64}:
 6.08696e5  9.13043e5
 2.6087e5   3.91304e5

In [41]:
det(MM)

0.9999974914225497

In [42]:
cond(MM, Inf)

1.9848821058834126e12

Returning to our $4096 \times 4096$ matrix, let's find its condition number

In [43]:
cond([F(i,j) for i=1:4096, j=1:4096], Inf)

2.40021656063533e6

Again, we'll learn more about what the matrix condition number means